In [0]:
Create a new notebook Load both csv and json file. Union them properly.
Instead of dropping the duplicates, create a boolean column is_duplicate and set False to only one row and set False to only one row and True to the rest of them

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df_csv= spark.read.format("csv").load("dbfs:/FileStore/tables/csv/batch.csv")
df_json= spark.read.format("json").load("dbfs:///FileStore/tables/json")

In [0]:
df_csv.show()

+---+-----+----------+----+------+----------+
|_c0|  _c1|       _c2| _c3|   _c4|       _c5|
+---+-----+----------+----+------+----------+
| id| name|       dob| age|salary|department|
|  1| John|1992-05-12|  30| 70000|        IT|
|  2|Alice|1997-02-28|  25| 60000|        HR|
|  3|  Bob|      null|null| 80000|        IT|
|  4|Emily|1994-11-22|  28| 65000|   Finance|
+---+-----+----------+----+------+----------+



In [0]:
df_json.show()

+----+----------+----------+---+------+------+
| age|department|       dob| id|  name|salary|
+----+----------+----------+---+------+------+
|  30|        IT|1992-05-12|  1|  John| 70000|
|  25|        HR|1997-02-28|  2| Alice| 60000|
|null|        IT|      null|  3|   Bob| 80000|
|  28|   Finance|1994-11-22|  4| Emily| 65000|
|  41|        HR|1981-12-18|  5| David| 90000|
|  33|   Finance|1989-07-05|  6| Susan| 75000|
|  46|        IT|1976-03-15|  7|  Mike| 95000|
|  30|   Finance|1992-06-30| 10|Sophie| 62000|
|  25|   Finance|1997-02-28|  2| Alice| 90000|
|  28|   Finance|1994-11-22|  4| Emily| 70000|
|  39|        IT|1983-10-14|  9| James| 87000|
|  30|        IT|1992-05-12|  1|  John| 70000|
|  27|        HR|1995-08-20|  8|  Lisa| 58000|
+----+----------+----------+---+------+------+



In [0]:
from pyspark.sql.types  import StructType, StructField, IntegerType,StringType,DateType

In [0]:
schema =StructType([
    StructField("id",IntegerType()),
    StructField("name",StringType()),
    StructField("dob",DateType()),
    StructField("age",IntegerType()),
    StructField("salary",IntegerType()),
    StructField("department",StringType()),
 
])
 

In [0]:
df_csv=spark.read.format("csv").schema(schema).option("header",True).load("dbfs:////FileStore/tables/csv/batch.csv")
df_json = spark.read.format("json").schema(schema).option("header",True).load("dbfs:///FileStore/tables/json")

In [0]:
# Task : Load both csv and json union them from pyspark.sql 

from pyspark.sql.window import Window
df=df_csv.union(df_json) 
display(df)

id,name,dob,age,salary,department
1,John,1992-05-12,30,70000,IT
2,Alice,1997-02-28,25,60000,HR
3,Bob,null,null,80000,IT
4,Emily,1994-11-22,28,65000,Finance
1,John,1992-05-12,30,70000,IT
2,Alice,1997-02-28,25,60000,HR
3,Bob,null,null,80000,IT
4,Emily,1994-11-22,28,65000,Finance
5,David,1981-12-18,41,90000,HR
6,Susan,1989-07-05,33,75000,Finance


In [0]:
from pyspark.sql import functions as F
df.withColumn( "is_duplicate", (F.row_number().over(Window.partitionBy(df.columns).orderBy("id"))) > 1 ).show()

+---+------+----------+----+------+----------+------------+
| id|  name|       dob| age|salary|department|is_duplicate|
+---+------+----------+----+------+----------+------------+
|  1|  John|1992-05-12|  30| 70000|        IT|       false|
|  1|  John|1992-05-12|  30| 70000|        IT|        true|
|  1|  John|1992-05-12|  30| 70000|        IT|        true|
|  2| Alice|1997-02-28|  25| 60000|        HR|       false|
|  2| Alice|1997-02-28|  25| 60000|        HR|        true|
|  2| Alice|1997-02-28|  25| 90000|   Finance|       false|
|  3|   Bob|      null|null| 80000|        IT|       false|
|  3|   Bob|      null|null| 80000|        IT|        true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|       false|
|  4| Emily|1994-11-22|  28| 65000|   Finance|        true|
|  4| Emily|1994-11-22|  28| 70000|   Finance|       false|
|  5| David|1981-12-18|  41| 90000|        HR|       false|
|  6| Susan|1989-07-05|  33| 75000|   Finance|       false|
|  7|  Mike|1976-03-15|  46| 95000|     

In [0]:
window =Window.partitionBy(*df.columns).orderBy("id")   
df.withColumn("is_duplicate", 
    F.row_number().over(window) != 1,      
).show()


+---+------+----------+----+------+----------+------------+
| id|  name|       dob| age|salary|department|is_duplicate|
+---+------+----------+----+------+----------+------------+
|  1|  John|1992-05-12|  30| 70000|        IT|       false|
|  1|  John|1992-05-12|  30| 70000|        IT|        true|
|  1|  John|1992-05-12|  30| 70000|        IT|        true|
|  2| Alice|1997-02-28|  25| 60000|        HR|       false|
|  2| Alice|1997-02-28|  25| 60000|        HR|        true|
|  2| Alice|1997-02-28|  25| 90000|   Finance|       false|
|  3|   Bob|      null|null| 80000|        IT|       false|
|  3|   Bob|      null|null| 80000|        IT|        true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|       false|
|  4| Emily|1994-11-22|  28| 65000|   Finance|        true|
|  4| Emily|1994-11-22|  28| 70000|   Finance|       false|
|  5| David|1981-12-18|  41| 90000|        HR|       false|
|  6| Susan|1989-07-05|  33| 75000|   Finance|       false|
|  7|  Mike|1976-03-15|  46| 95000|     

In [0]:
Tasks:
Calculate mean salary and check if it is greater or equal to the salary of employees in each department.

Calculate mean salary and check if it is greater or equal to the salary of all employees.

In [0]:
department_window =Window.partitionBy("department")


In [0]:
df.withColumns(
    {
        "mean_salary":F.mean("salary").over(department_window),
        "is_salary_greater":F.col("salary") >= F.col("mean_salary"),
    }
).show()

+---+------+----------+----+------+----------+-----------------+-----------------+
| id|  name|       dob| age|salary|department|      mean_salary|is_salary_greater|
+---+------+----------+----+------+----------+-----------------+-----------------+
|  4| Emily|1994-11-22|  28| 65000|   Finance|71166.66666666667|            false|
|  4| Emily|1994-11-22|  28| 65000|   Finance|71166.66666666667|            false|
|  6| Susan|1989-07-05|  33| 75000|   Finance|71166.66666666667|             true|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|71166.66666666667|            false|
|  2| Alice|1997-02-28|  25| 90000|   Finance|71166.66666666667|             true|
|  4| Emily|1994-11-22|  28| 70000|   Finance|71166.66666666667|            false|
|  2| Alice|1997-02-28|  25| 60000|        HR|          67000.0|            false|
|  2| Alice|1997-02-28|  25| 60000|        HR|          67000.0|            false|
|  5| David|1981-12-18|  41| 90000|        HR|          67000.0|             true|
|  8

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

window_spec = Window.partitionBy("department")

result_df = df.withColumn("mean_salary", F.mean("salary").over(window_spec)) \
              .withColumn("above_mean_salary", F.col("salary") >= F.mean("salary").over(window_spec))

result_df.show()



+---+------+----------+----+------+----------+-----------------+-----------------+
| id|  name|       dob| age|salary|department|      mean_salary|above_mean_salary|
+---+------+----------+----+------+----------+-----------------+-----------------+
|  4| Emily|1994-11-22|  28| 65000|   Finance|71166.66666666667|            false|
|  4| Emily|1994-11-22|  28| 65000|   Finance|71166.66666666667|            false|
|  6| Susan|1989-07-05|  33| 75000|   Finance|71166.66666666667|             true|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|71166.66666666667|            false|
|  2| Alice|1997-02-28|  25| 90000|   Finance|71166.66666666667|             true|
|  4| Emily|1994-11-22|  28| 70000|   Finance|71166.66666666667|            false|
|  2| Alice|1997-02-28|  25| 60000|        HR|          67000.0|            false|
|  2| Alice|1997-02-28|  25| 60000|        HR|          67000.0|            false|
|  5| David|1981-12-18|  41| 90000|        HR|          67000.0|             true|
|  8

In [0]:
window_spec = Window.partitionBy("department")

df.withColumn(
     "mean_salary_department",
     F.col("salary") >= F.mean("salary").over(window_spec)
 ).show()

+---+------+----------+----+------+----------+----------------------+
| id|  name|       dob| age|salary|department|mean_salary_department|
+---+------+----------+----+------+----------+----------------------+
|  4| Emily|1994-11-22|  28| 65000|   Finance|                 false|
|  4| Emily|1994-11-22|  28| 65000|   Finance|                 false|
|  6| Susan|1989-07-05|  33| 75000|   Finance|                  true|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|                 false|
|  2| Alice|1997-02-28|  25| 90000|   Finance|                  true|
|  4| Emily|1994-11-22|  28| 70000|   Finance|                 false|
|  2| Alice|1997-02-28|  25| 60000|        HR|                 false|
|  2| Alice|1997-02-28|  25| 60000|        HR|                 false|
|  5| David|1981-12-18|  41| 90000|        HR|                  true|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|                 false|
|  1|  John|1992-05-12|  30| 70000|        IT|                 false|
|  3|   Bob|      nu

In [0]:
average_salary_by_department = df.groupBy("department").agg(
    F.avg("salary").alias("average_salary")
)


In [0]:
average_salary_by_department.show()

+----------+-----------------+
|department|   average_salary|
+----------+-----------------+
|        HR|          67000.0|
|   Finance|71166.66666666667|
|        IT|78857.14285714286|
+----------+-----------------+



In [0]:

window_spec =Window.partitionBy()

df.withColumns(
    {
        "mean": F.mean("salary").over(window_spec),
        "is_salary_above_mean":
        F.col("salary")>=F.mean("salary").over(window_spec)
    }
).show()

+---+------+----------+----+------+----------+-----------------+--------------------+
| id|  name|       dob| age|salary|department|             mean|is_salary_above_mean|
+---+------+----------+----+------+----------+-----------------+--------------------+
|  1|  John|1992-05-12|  30| 70000|        IT|73352.94117647059|               false|
|  2| Alice|1997-02-28|  25| 60000|        HR|73352.94117647059|               false|
|  3|   Bob|      null|null| 80000|        IT|73352.94117647059|                true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|73352.94117647059|               false|
|  1|  John|1992-05-12|  30| 70000|        IT|73352.94117647059|               false|
|  2| Alice|1997-02-28|  25| 60000|        HR|73352.94117647059|               false|
|  3|   Bob|      null|null| 80000|        IT|73352.94117647059|                true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|73352.94117647059|               false|
|  5| David|1981-12-18|  41| 90000|        HR|73352.94

In [0]:
window_spec = Window.partitionBy()
window_spec = Window.orderBy()
window_spec = Window.partitionBy().orderBy()

df.withColumn(
    "is_salary_above_mean",
    (
        F.col("salary") >= F.mean("salary").over(window_spec)
    )
).show()

+---+------+----------+----+------+----------+--------------------+
| id|  name|       dob| age|salary|department|is_salary_above_mean|
+---+------+----------+----+------+----------+--------------------+
|  1|  John|1992-05-12|  30| 70000|        IT|               false|
|  2| Alice|1997-02-28|  25| 60000|        HR|               false|
|  3|   Bob|      null|null| 80000|        IT|                true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|               false|
|  1|  John|1992-05-12|  30| 70000|        IT|               false|
|  2| Alice|1997-02-28|  25| 60000|        HR|               false|
|  3|   Bob|      null|null| 80000|        IT|                true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|               false|
|  5| David|1981-12-18|  41| 90000|        HR|                true|
|  6| Susan|1989-07-05|  33| 75000|   Finance|                true|
|  7|  Mike|1976-03-15|  46| 95000|        IT|                true|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|  